In [275]:
import numpy as np

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling1D, MaxPooling2D, LeakyReLU
from keras.initializers import RandomNormal
import keras.optimizers as opt
from keras import regularizers
from keras.callbacks import TensorBoard
from keras.utils import plot_model

In [276]:
data = np.load('../data/data.npy')
meta = np.load('../data/meta.npy')
print(f'data = {data.shape}')
print(f'meta = {meta.shape}')

data = (1850, 21, 77)
meta = (1850, 3)


In [277]:
X = data[:,:,:,np.newaxis]
Y = meta[:,2][:,np.newaxis].astype(int)

In [278]:
X,Y = shuffle(X,Y)

In [279]:
x_train, x_test,  y_train, y_test  = train_test_split(X, Y, test_size = 0.2, random_state = 31)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.25, random_state = 31)

In [308]:
np.unique(y_valid,return_counts=True)


(array([0, 1]), array([251, 119]))

In [280]:
print(f"train: x={x_train.shape} y={y_train.shape}")
print(f"valid: x={x_valid.shape}  y={y_valid.shape}")
print(f"test : x={x_test.shape}  y={y_test.shape}")

train: x=(1110, 21, 77, 1) y=(1110, 1)
valid: x=(370, 21, 77, 1)  y=(370, 1)
test : x=(370, 21, 77, 1)  y=(370, 1)


In [281]:
cnn_input = X.shape[1:]
cnn_output = Y.shape[1]
batch_size = 32

print(f"cnn_input  = {cnn_input}")
print(f"cnn_output = {cnn_output}")
print(f"batch_size = {batch_size}")

cnn_input  = (21, 77, 1)
cnn_output = 1
batch_size = 32


In [327]:
model = Sequential()

model.add(Conv2D(16, (7,7), activation = 'relu', name = 'conv1', input_shape = cnn_input))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), name='maxpool1'))

model.add(Conv2D(32, (5,5), activation='relu', name='conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), name='maxpool2'))

# model.add(Conv2D(64, (3,3), activation='relu', name='conv3'))
# model.add(MaxPooling2D(pool_size=(2,2), name='maxpool3'))

model.add(Flatten(name='flat4'))
model.add(Dense(1024, activation='relu', name='dense4'))
model.add(Dropout(0.5, name='drop4'))

model.add(Dense(cnn_output, activation='sigmoid', name='softmax'))

In [328]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 15, 71, 16)        800       
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 14, 70, 16)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 10, 66, 32)        12832     
_________________________________________________________________
maxpool2 (MaxPooling2D)      (None, 9, 65, 32)         0         
_________________________________________________________________
flat4 (Flatten)              (None, 18720)             0         
_________________________________________________________________
dense4 (Dense)               (None, 1024)              19170304  
_________________________________________________________________
drop4 (Dropout)              (None, 1024)              0         
__________

In [329]:
optim = opt.SGD(lr = 0.001, decay = 1e-6, momentum = 0.5, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])

In [330]:
model.fit(x_train, y_train, batch_size = batch_size, epochs = 100, 
          verbose = 2, validation_data=(x_valid, y_valid))

Train on 1110 samples, validate on 370 samples
Epoch 1/100
 - 2s - loss: 1.1820 - acc: 0.6649 - val_loss: 0.6867 - val_acc: 0.6784
Epoch 2/100
 - 0s - loss: 0.6824 - acc: 0.6883 - val_loss: 0.6793 - val_acc: 0.6784
Epoch 3/100
 - 0s - loss: 0.6752 - acc: 0.6883 - val_loss: 0.6714 - val_acc: 0.6784
Epoch 4/100
 - 0s - loss: 0.6652 - acc: 0.6883 - val_loss: 0.6623 - val_acc: 0.6784
Epoch 5/100
 - 0s - loss: 0.6554 - acc: 0.6883 - val_loss: 0.6527 - val_acc: 0.6784
Epoch 6/100
 - 0s - loss: 0.6450 - acc: 0.6883 - val_loss: 0.6435 - val_acc: 0.6784
Epoch 7/100
 - 0s - loss: 0.6362 - acc: 0.6883 - val_loss: 0.6365 - val_acc: 0.6784
Epoch 8/100
 - 0s - loss: 0.6302 - acc: 0.6883 - val_loss: 0.6319 - val_acc: 0.6784
Epoch 9/100
 - 0s - loss: 0.6251 - acc: 0.6883 - val_loss: 0.6295 - val_acc: 0.6784
Epoch 10/100
 - 0s - loss: 0.6240 - acc: 0.6883 - val_loss: 0.6286 - val_acc: 0.6784
Epoch 11/100
 - 0s - loss: 0.6211 - acc: 0.6883 - val_loss: 0.6282 - val_acc: 0.6784
Epoch 12/100
 - 0s - loss: 

Epoch 97/100
 - 0s - loss: 0.6217 - acc: 0.6883 - val_loss: 0.6283 - val_acc: 0.6784
Epoch 98/100
 - 0s - loss: 0.6215 - acc: 0.6883 - val_loss: 0.6283 - val_acc: 0.6784
Epoch 99/100
 - 0s - loss: 0.6206 - acc: 0.6883 - val_loss: 0.6284 - val_acc: 0.6784
Epoch 100/100
 - 0s - loss: 0.6221 - acc: 0.6883 - val_loss: 0.6284 - val_acc: 0.6784


In [332]:
for i in zip(model.metrics_names, model.evaluate(x_test, y_test, batch_size = 32)):
    print(i)

370/370 [==============================] - 0s 92us/step
('loss', 0.6111326601054218)
('acc', 0.6999999996778127)


In [335]:
model.predict(x_test)

array([[0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.31068683],
       [0.310